## Imports


In [53]:
import numpy as np
from icecream import ic
from gxgp import  Node
from tqdm import tqdm
import operator
import inspect

In [54]:
a = np.array([1])

In [55]:
def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5

In [78]:
def my_f(x: np.ndarray) -> np.ndarray:
    return x[0]

In [83]:
TEST_SIZE = 10_000
TRAIN_SIZE = 1000

def gen_problem():
    x_validation = np.vstack(
        [
            np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
            np.random.random_sample(size=TEST_SIZE) * 2 - 1,
        ]
    )
    y_validation = true_f(x_validation)
    train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
    x_train = x_validation[:, train_indexes]
    y_train = y_validation[train_indexes]
    assert np.all(y_train == true_f(x_train)), "D'ho"
    return x_train, y_train

# np.savez('problem_0.npz', x=x_train, y=y_train)
problem = np.load('problem_0.npz')
x = problem['x']
y = problem['y']
x.shape[0]
number_vars = x.shape[0]
# compute mse with truef as reference
mse = true_f(x) - my_f(x)
x



array([[-2.4300303 ,  2.38282117,  1.38014952, ...,  2.65852815,
        -2.72159251,  1.4719692 ],
       [-0.40491585,  0.62268698, -0.52027988, ..., -0.55608775,
         0.60177422, -0.44432087]])

### Getting numpy functions

In [58]:
def get_np_functions()->list[tuple[np.ufunc,str,int]]:
    # Get all attributes of numpy module
    all_attrs = dir(np)

    # Filter to include only ufunc objects
    ufuncs = [getattr(np, attr) for attr in all_attrs if isinstance(getattr(np, attr), np.ufunc)]
    # Filter to include only common ufuncs
    # ufunc_list = [(ufunc,ufunc.__name__,ufunc.nin) for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    ufunc_list = [ufunc for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    return ufunc_list
def myf(x: np.ndarray) -> np.ndarray:
    return np.subtract(np.subtract(np.minimum(np.multiply(x[0],x[1]),np.multiply(x[2],x[3])),np.maximum(np.subtract(x[4],x[5]),np.multiply(x[6],x[7]))),np.divide(np.maximum(np.exp2(x[8]),np.subtract(x[9],69)),np.divide(np.log(32),np.cos(97))))
myf(np.array([1,2,3,4,5,6,7,9,9,9,9]))
s = np.array([1,2,3])
np.concatenate([s,[None,None]])
np.array([None] * (5 + 1))


array([None, None, None, None, None, None], dtype=object)

## Symbolic Regressor


### Utilify Functions


In [ ]:
from tree4 import FastTBTree
def RHH_Init():
    pass
def fitness(tree:FastTBTree,x_train:np.ndarray,y_train:np.ndarray)->float:
    res = [tree.evalute_tree(x) for x in x_train.T]
    return np.mean((res-y_train)**2)
def fitness_proportional_selection(trees:list[FastTBTree],n_elems=2)->list[FastTBTree]:
    """
    Windowing version of fitness proportional selection
    Each tree has a probability of (fitness-min(fitnesses))/(sum(fitnesses)-min(fitnesses)) to be chosen
    """
    fitnesses = np.array([tree.fitness for tree in trees])
    # print(fitnesses)
    fitnesses = fitnesses - np.min(fitnesses)
    fitnesses = fitnesses / np.sum(fitnesses)
    # replace NaN with an high value
    return np.random.choice(trees, n_elems, p=fitnesses)
def tournament_selection(trees:list[FastTBTree],n_elems=2)->list[FastTBTree]:
    """
    Tournament selection
    """
    challengers = np.random.choice(trees, n_elems)
    return sorted(challengers, key=lambda x: x.fitness)[0]


def mutation(tree:FastTBTree,max_const:int,md:int)->FastTBTree:
    """
    Mutation of a tree
    Randomly select a mutation method and apply it
    """
    mutation_kind = np.random.randint(5)
    match mutation_kind:
        case 0:
            tree.point_mutation(max_const=max_const)
            return tree
        case 1:
            tree.expansion_mutation(md)
            return tree
        case 2:
            tree.hoist_mutation()
            return tree
        case 3:
            tree.subtree_mutation(md)
            return tree
        case 4:
            tree.collapse_subtree_mutation()
            return tree




### Regression 

In [ ]:
N_VARS = number_vars
MD = 5 # Maximum depth
MAX_CONST = 1000 # Maximum constant value
OFFSPRING_SIZE = 10
GENERATIONS = 40
PM = 0.05
NAN_PENALTY = 10000

operator_list = get_np_functions()
FastTBTree.set_params(operator_list,N_VARS,MAX_CONST)
# population = [TweakableBTree.generate_random_tree_growfull(operator_list,N_VARS,MD,True,PM) for _ in range(OFFSPRING_SIZE)]
population = [FastTBTree.generate_random_tree_growfull(True,MD) for _ in range(OFFSPRING_SIZE)]

for elem in population:
    elem.fitness = fitness(elem,x,y)
    if np.isnan(elem.fitness) or np.isinf(elem.fitness):
        elem.fitness = np.float64(NAN_PENALTY)
    # ic(elem.fitness)
offspring = []
for generation in tqdm(range(GENERATIONS)):
    # parent selection
    # parents = fitness_proportional_selection(population)
    parents = [tournament_selection(population) for _ in range(2)]

    # recombination
    for _ in range(OFFSPRING_SIZE):
        # o1,o2 = recombination(parents[0],parents[1])
        o1,o2 = FastTBTree.recombination(parents[0],parents[1])
        offspring.append(o1)
        offspring.append(o2)

    # if np.random.rand() < PM:
    #     offspring_idx = np.random.randint(OFFSPRING_SIZE)
    #     mutation(population[offspring_idx],MAX_CONST,MD)

    # mutation
    

    # survivor selection
    population.extend(offspring)
    for elem in population:
        elem.fitness = fitness(elem,x,y)
        if np.isnan(elem.fitness)  or np.isinf(elem.fitness):
            elem.fitness = np.float64(NAN_PENALTY)
    population.sort(key=operator.attrgetter('fitness'))
    population = population[:OFFSPRING_SIZE]

# ic([elem for elem in population if not np.isnan(elem.fitness) and not  np.isinf(elem.fitness)])

c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: invalid value encountered in sqrt
  return node.data(FastTBTree._evalute_tree(tree, FastTBTree.get_lchild_idx(i), x))
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: invalid value encountered in log2
  return node.data(FastTBTree._evalute_tree(tree, FastTBTree.get_lchild_idx(i), x))
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: overflow encountered in cosh
  return node.data(FastTBTree._evalute_tree(tree, FastTBTree.get_lchild_idx(i), x))
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: invalid value encountered in log
  return node.data(FastTBTree._evalute_tree(tree, FastTBTree.get_lchild_idx(i), x))
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree4.py:209: RuntimeWarning: overflow encountered in sinh
  return node.data(FastTBTree._evalute_tree(tree, FastT

In [1]:
ic([elem.fitness for elem in population])
# ic([elem.to_np_formula() for elem in population])
population[1].print_tree()


NameError: name 'ic' is not defined